# Modeling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, mean_absolute_error, mean_squared_error, r2_score

from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression

Load Combined Weather/Flight Data

In [2]:
file_path = r"/Users/kianmohseni/Documents/DSC288R_Capstone/data/final/final_data.parquet"

df = pd.read_parquet(file_path)

In [3]:
print(df.columns)

Index(['FlightDate', 'DayOfWeek', 'Month', 'Airline', 'Origin', 'Dest',
       'Cancelled', 'Diverted', 'DivAirportLandings', 'CRSDepTime', 'DepTime',
       'DepDelay', 'DepDel15', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDel15',
       'AirTime', 'Distance', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn',
       'DelayCategory', 'AirTimeCategory', 'TimeofDay', 'Origin_PRCP',
       'Origin_SNOW', 'Origin_SNWD', 'Origin_TMAX', 'Origin_TMIN', 'Dest_PRCP',
       'Dest_SNOW', 'Dest_SNWD', 'Dest_TMAX', 'Dest_TMIN'],
      dtype='object')


In [4]:
df["AirportCongestion"] = df.groupby(["Origin", "DayOfWeek"])["DepDelay"].transform("count")
df["WeatherRiskScore"] = df["Origin_PRCP"] + df["Dest_PRCP"] + df["Origin_SNOW"] + df["Dest_SNOW"] + df["Origin_SNWD"] + df["Dest_SNWD"]

In [5]:
# Get U.S. Holidays dynamically
us_holidays = holidays.US(years=[2018, 2019, 2020, 2021, 2022]) 

# Convert to a list of dates
holiday_dates_dynamic = pd.to_datetime(list(us_holidays.keys()))

# Create a Holiday Indicator (1 if flight is on a holiday, else 0)
df['HolidayIndicator'] = df['FlightDate'].isin(holiday_dates_dynamic).astype(int)

Split into Train & Test Sets

In [6]:
# Define feature set and targets
X = df.drop(columns=['DayOfWeek', 'DelayCategory', 'AirTime', 'Origin_PRCP',
       'Origin_SNOW', 'Origin_SNWD', 'Dest_PRCP',
       'Dest_SNOW', 'Dest_SNWD', 'ArrDel15', 'Airline', 'Origin', 'Dest', 'DepDelay', 'ArrDelay', 'DepDel15'])
y_classification = df['DepDel15']
y_regression = df['DepDelay']

In [7]:
# Scale numerical data
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

In [8]:
# Drop original FlightDate column since it's not numerical
X = X.drop(columns=['FlightDate'])

In [9]:
# Label encode the column
encoder = LabelEncoder()
X['AirTimeCategory'] = encoder.fit_transform(X['AirTimeCategory'])
X['TimeofDay'] = encoder.fit_transform(X['TimeofDay'])


In [10]:
X = X.fillna(X.median())  # Fill NaN with median
y_classification = y_classification.fillna(y_classification.median())  # Fill NaN with median
y_regression = y_regression.fillna(y_regression.median())  # Fill NaN with median

In [11]:
# Feature selection
k = 10  # number of top features to select
selector = SelectKBest(score_func = f_regression, k = k)
X_selected_classification = selector.fit_transform(X, y_classification)

selected_features = X.columns[selector.get_support()].tolist()
print("Selected features:", selected_features)

Selected features: ['Cancelled', 'CRSDepTime', 'DepTime', 'CRSArrTime', 'ArrTime', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TimeofDay', 'WeatherRiskScore']


In [12]:
# Feature selection
k = 10  # number of top features to select
selector = SelectKBest(score_func = f_regression, k = k)
X_selected_regression = selector.fit_transform(X, y_regression)

selected_features = X.columns[selector.get_support()].tolist()
print("Selected features:", selected_features)

Selected features: ['Cancelled', 'CRSDepTime', 'DepTime', 'CRSArrTime', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'AirTimeCategory', 'TimeofDay', 'WeatherRiskScore']


In [13]:
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_selected_classification, y_classification, test_size=0.2, random_state=42, stratify=y_classification)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_selected_regression, y_regression, test_size=0.2, random_state=42)

K-fold Cross Validation

In [14]:
k = 5

# Stratified K-Fold for Classification
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Regular K-Fold for Regression
kf = KFold(n_splits=k, shuffle=True, random_state=42)

In [15]:
# Undersampling (RUS)
rus = RandomUnderSampler(random_state=42)
X_resampled_rus_class, y_resampled_rus_class = rus.fit_resample(X_selected_classification, y_classification)


In [16]:
# Undersampling (RUS)
rus = RandomUnderSampler(random_state=42)
X_resampled_rus_reg, y_resampled_rus_reg = rus.fit_resample(X_selected_regression, y_regression)

## Classification

Random Forest

In [17]:
# clf = RandomForestClassifier(random_state=42)
# classification_scores = cross_val_score(clf, X_resampled_rus_class, y_resampled_rus_class, cv=skf, scoring='accuracy')
# print(f"Random Forest Classification Accuracy Scores: {classification_scores}")
# print(f"Mean Accuracy: {np.mean(classification_scores):.4f}")

## Regression

Linear Regression

In [18]:
lin_reg = LinearRegression()
regression_scores = cross_val_score(lin_reg, X_resampled_rus_reg, y_resampled_rus_reg, cv=kf, scoring='r2')
print(f"Multi-Linear Regression R^2 Scores: {regression_scores}")
print(f"Mean R^2 Score: {np.mean(regression_scores):.4f}")


Multi-Linear Regression R^2 Scores: [0.08435697 0.04317186 0.05102737 0.06589766 0.08812158]
Mean R^2 Score: 0.0665


In [22]:
# Ensure no zero or negative values before log transformation
y_log = np.log1p(y_resampled_rus_reg - y_resampled_rus_reg.min() + 1)

# Handle NaN and infinite values
y_log.replace([np.inf, -np.inf], np.nan, inplace=True)  # Convert infinities to NaN
y_log.dropna(inplace=True)  # Drop rows with NaN

# Fit linear regression model
lin_reg = LinearRegression()
regression_scores = cross_val_score(lin_reg, X_resampled_rus_reg, y_log, cv=kf, scoring='r2')

print(f"Multi-Linear Regression (Log Transformed) R^2 Scores: {regression_scores}")
print(f"Mean R^2 Score: {np.mean(regression_scores):.4f}")

Multi-Linear Regression (Log Transformed) R^2 Scores: [0.08616527 0.05386638 0.02599544 0.07246875 0.08038498]
Mean R^2 Score: 0.0638


In [24]:
# Ensure all values are non-negative before sqrt transformation
y_sqrt = np.sqrt(y_resampled_rus_reg - y_resampled_rus_reg.min() + 1)

# Handle NaN and infinite values
y_sqrt.replace([np.inf, -np.inf], np.nan, inplace=True)  # Convert infinities to NaN
y_sqrt.dropna(inplace=True)  # Drop rows with NaN

# Fit model
lin_reg = LinearRegression()
regression_scores = cross_val_score(lin_reg, X_resampled_rus_reg, y_sqrt, cv=kf, scoring='r2')

print(f"Multi-Linear Regression (Square Root Transformed) R^2 Scores: {regression_scores}")
print(f"Mean R^2 Score: {np.mean(regression_scores):.4f}")

Multi-Linear Regression (Square Root Transformed) R^2 Scores: [0.08780061 0.04948788 0.05124501 0.07313246 0.08661493]
Mean R^2 Score: 0.0697


In [28]:
# Step 1: Shift values to avoid division by zero
y_shifted = y_resampled_rus_reg.copy()

# Replace zeros and small values with a small positive number
y_shifted[y_shifted <= 1e-5] = 1e-5  

# Step 2: Apply logarithm first to stabilize variance
y_log = np.log1p(y_shifted)  # log(1 + x) prevents log(0) issues

# Step 3: Apply reciprocal transformation
y_reciprocal = 1 / y_log  # Applying reciprocal to the smoothed values

# Step 4: Scale using StandardScaler (better for regression)
scaler = StandardScaler()
y_reciprocal_scaled = scaler.fit_transform(y_reciprocal.values.reshape(-1, 1)).flatten()

# Step 5: Handle NaN and infinite values
if np.isnan(y_reciprocal_scaled).sum() > 0 or np.isinf(y_reciprocal_scaled).sum() > 0:
    y_reciprocal_scaled = np.nan_to_num(y_reciprocal_scaled, nan=np.median(y_reciprocal_scaled))

# Fit model
lin_reg = LinearRegression()
regression_scores = cross_val_score(lin_reg, X_resampled_rus_reg, y_reciprocal_scaled, cv=kf, scoring='r2')

print(f"Multi-Linear Regression (Reciprocal Transformed) R^2 Scores: {regression_scores}")
print(f"Mean R^2 Score: {np.mean(regression_scores):.4f}")

Multi-Linear Regression (Reciprocal Transformed) R^2 Scores: [ 1.79201922e-02  1.17056279e-02 -1.04923937e-02 -6.55229890e-05
 -1.64035670e-02]
Mean R^2 Score: 0.0005


: 

In [26]:
# Create polynomial features
poly = PolynomialFeatures(degree=2)  # Quadratic transformation
X_poly = poly.fit_transform(X_resampled_rus_reg)

# Fit model
lin_reg = LinearRegression()
regression_scores = cross_val_score(lin_reg, X_poly, y_resampled_rus_reg, cv=kf, scoring='r2')

print(f"Multi-Linear Regression (Polynomial Transformed) R^2 Scores: {regression_scores}")
print(f"Mean R^2 Score: {np.mean(regression_scores):.4f}")

Multi-Linear Regression (Polynomial Transformed) R^2 Scores: [0.11264245 0.04469127 0.03264506 0.07862475 0.08293496]
Mean R^2 Score: 0.0703


## Model Evaluation

In [ ]:
# # Store classification results
# classification_results = {
#     "Random Forest Classifier": accuracy_score(y_test_class, y_pred_rf),
# }

# # Store regression results
# regression_results = {
#     "Linear Regression": mean_absolute_error(y_test_reg, y_pred_linreg),
# }

# print("\nClassification Model Accuracy:")
# for model, acc in classification_results.items():
#     print(f"{model}: {acc:.4f}")

# print("\nRegression Model Mean Absolute Error (Lower is better):")
# for model, mae in regression_results.items():
#     print(f"{model}: {mae:.2f}")